# Advent of code 2020: day 16

Problem [here](https://adventofcode.com/2020/day/16)

## Part 1

In [1]:
ex_input = """class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12"""

class Field:
    def __init__(self, validRanges):
        self.validRanges = validRanges
    def maybeValid(self, num):
        return any(lw <= num and num <= up for lw, up in self.validRanges)
    @staticmethod
    def parse(spec):
        return Field([ tuple(int(vl) for vl in tok.split("-")) for tok in spec.split(" or ") ])
    def __str__(self):
        return " or ".join("-".join(rng) for rng in self.validRanges)

def parseNotes(lines):
    iLines = iter(lines)
    fields, mine, nearby = {}, None, []
    while ln := next(iLines): ## read fields
        fNm, opts = ln.split(": ")
        fields[fNm] = Field.parse(opts)
    assert next(iLines) == "your ticket:"
    mine = [ int(val) for val in next(iLines).split(",") ]
    assert not next(iLines) ## skip empty
    assert next(iLines) == "nearby tickets:"
    try:
        while ln := next(iLines): ## read nearby
            nearby.append([ int(val) for val in ln.split(",") ])
    except StopIteration:
        pass
    return fields, mine, nearby

ex_fields, ex_mine, ex_nearby = parseNotes(ex_input.split("\n"))
print(ex_fields)
print(ex_mine)
print(ex_nearby)
ex_errRate = 0
for nbTk in ex_nearby:
    for val in nbTk:
        if not any(field.maybeValid(val) for field in ex_fields.values()):
            print(f"Value {val:d} in nearby ticket {nbTk!r} is invalid")
            ex_errRate += val
print(f"Example scanning error rate: {ex_errRate:d}")

{'class': <__main__.Field object at 0x7ffb98a2eb50>, 'row': <__main__.Field object at 0x7ffb98a2e910>, 'seat': <__main__.Field object at 0x7ffb98a2ea90>}
[7, 1, 14]
[[7, 3, 47], [40, 4, 50], [55, 2, 20], [38, 6, 12]]
Value 4 in nearby ticket [40, 4, 50] is invalid
Value 55 in nearby ticket [55, 2, 20] is invalid
Value 12 in nearby ticket [38, 6, 12] is invalid
Example scanning error rate: 71


In [2]:
with open("inputs/day16.txt") as inF:
    p1_fields, p1_mine, p1_nearby = parseNotes(( ln.strip() for ln in inF ))
    
p1_errRate = 0
valid_nearby = []
for nbTk in p1_nearby:
    isValid = True
    for val in nbTk:
        if not any(field.maybeValid(val) for field in p1_fields.values()):
            p1_errRate += val
            isValid = False
    if isValid:
        valid_nearby.append(nbTk)
print(f"Puzzle scanning error rate: {p1_errRate:d}")

Puzzle scanning error rate: 24110


## Part 2

In [3]:
compat_fields_per_col = [
    [ fNm for fNm,f in p1_fields.items() if all(f.maybeValid(nbTk[iCol]) for nbTk in valid_nearby ) ]
    for iCol in range(len(p1_fields)) ]
print([len(compat) for compat in compat_fields_per_col])
colNames = [ None for f in p1_fields ]
while any(cn is None for cn in colNames):
    assigned = []
    for iCol, compat in enumerate(compat_fields_per_col):
        if len(compat) == 1:
            name = compat[0]
            colNames[iCol] = name
            assigned.append(name)
    if not assigned:
        print("I'm stuck :-(")
        break
    assert len(assigned) == len(set(assigned))
    for compat in compat_fields_per_col:
        for name in assigned:
            if name in compat:
                del compat[compat.index(name)]
print(colNames)
depVals = [ mnVal for mnVal,colNm in zip(p1_mine, colNames) if colNm.startswith("departure") ]
print(depVals)
prod = 1
for v in depVals:
    prod *= v
print(prod)

[5, 14, 3, 2, 18, 13, 11, 10, 19, 15, 20, 17, 12, 8, 7, 6, 4, 1, 9, 16]
['arrival location', 'class', 'route', 'train', 'arrival platform', 'type', 'departure station', 'departure track', 'row', 'seat', 'arrival track', 'zone', 'departure date', 'departure platform', 'departure time', 'duration', 'price', 'wagon', 'departure location', 'arrival station']
[131, 239, 109, 73, 157, 173]
6766503490793
